In [9]:
# Imports
import pandas as pd
import math
import time 

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split

In [10]:
# Constants
DATA_PATH = './data/'
DATASET_PATH = DATA_PATH + 'uci-data/'
MODELS_PATH = DATA_PATH + 'models/filtered-models/'
FEATURES = ['accX', 'accY', 'accZ', 'gyroX', 'gyroY', 'gyroZ', "subject", "activity"]
SAMPLE_SIZE = 50000
BATCH_SIZE = 32

In [11]:
complete_set = pd.read_csv(DATA_PATH + "self-calculated/median-filtered.txt", sep='\s+', header=None)
complete_set.columns = FEATURES
train_set, test_set = train_test_split(complete_set, test_size=0.3)

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
/var/folders/lt/w0169b7x5ml3psz3nly9vj3m0000gn/T/ipykernel_36795/1398839783.py:1: SyntaxWarning: invalid escape sequence '\s'
  complete_set = pd.read_csv(DATA_PATH + "self-calculated/median-filtered.txt", sep='\s+', header=None)


In [12]:
class RawDataModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(len(FEATURES) - 1, 128)  # Adjust input size based on output of Conv1d
        self.relu1 = nn.ReLU()
        self.batchnorm1 = nn.BatchNorm1d(128)
        self.dropout1 = nn.Dropout(0.05)

        self.fc2 = nn.Linear(128, 64)  # Adjust input size based on output of Conv1d
        self.relu2 = nn.ReLU()
        self.batchnorm2 = nn.BatchNorm1d(64)
        self.dropout2 = nn.Dropout(0.05)

        self.fc3 = nn.Linear(64, 32)  # Adjust input size based on output of Conv1d
        self.relu3 = nn.ReLU()
        self.batchnorm3 = nn.BatchNorm1d(32)
        self.dropout3 = nn.Dropout(0.05)
        
        self.fc4 = nn.Linear(32, 16)
        self.silu = nn.SiLU()
        self.fc5 = nn.Linear(16, 12)
        self.softmax = nn.Softmax(dim=1)  # Use dim=1 for multi-class classification

    def forward(self, x):
        x = self.fc1(x)           # Shape: [batch_size, 512]
        x = self.relu1(x)         # Apply ReLU activation
        x = self.batchnorm1(x)
        x = self.dropout1(x)       # Apply dropout

        x = self.fc2(x)           # Shape: [batch_size, 512]
        x = self.relu2(x)         # Apply ReLU activation
        x = self.batchnorm2(x)
        x = self.dropout2(x)       # Apply dropout

        x = self.fc3(x)           # Shape: [batch_size, 512]
        x = self.relu3(x)         # Apply ReLU activation
        x = self.batchnorm3(x)
        x = self.dropout3(x)       # Apply dropout

        x = self.fc4(x)           # Shape: [batch_size, 32]
        x = self.silu(x)          # Apply SiLU activation
        x = self.fc5(x)           # Shape: [batch_size, 8]
        x = self.softmax(x)       # Apply Softmax for classification
        
        return x

In [13]:
# Training and testing function
def train(dataloader, model, loss_fn, optimizer):
    # Get batch num
    num_batches = len(dataloader.dataset) / BATCH_SIZE
    i = 0

    # Set the model to train mode
    model.train()

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        i += 1
        if batch % 50 == 0:
            print(f"loss: {loss.item()}, batch: {i} out of {math.ceil(num_batches)}")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    
    return(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [14]:
# Dataset preparation
class HAPTDataset(Dataset):
    def __init__(self, dataset, features, label):
        self.indexes = torch.randperm(dataset.shape[0])
        self.data = torch.tensor(dataset[features].values, dtype=torch.float32)[:SAMPLE_SIZE]
        self.labels = torch.tensor(dataset[label].values, dtype=torch.float32)[:SAMPLE_SIZE]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]
    
train_dataset = HAPTDataset(train_set, train_set.columns[:-1], 'activity')
test_dataset = HAPTDataset(test_set, test_set.columns[:-1], 'activity')

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [15]:
epochs = 5
perf_timer = time.perf_counter()
perf_acc = ""

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_function, optimizer)
    perf_acc = test(test_dataloader, model, loss_function)
    
print(perf_acc)
perf_timer = time.perf_counter() - perf_timer
print("Done!")

Epoch 1
-------------------------------
loss: 1.9174268245697021, batch: 1 out of 1563
loss: 1.726719617843628, batch: 51 out of 1563
loss: 1.8221793174743652, batch: 101 out of 1563
loss: 1.9826693534851074, batch: 151 out of 1563
loss: 1.850569725036621, batch: 201 out of 1563
loss: 1.7433148622512817, batch: 251 out of 1563
loss: 1.9115591049194336, batch: 301 out of 1563
loss: 1.7983553409576416, batch: 351 out of 1563
loss: 1.9128297567367554, batch: 401 out of 1563
loss: 1.8828799724578857, batch: 451 out of 1563
loss: 1.8310587406158447, batch: 501 out of 1563
loss: 1.92349374294281, batch: 551 out of 1563
loss: 1.882470965385437, batch: 601 out of 1563
loss: 1.9217474460601807, batch: 651 out of 1563
loss: 1.6738560199737549, batch: 701 out of 1563
loss: 1.9570955038070679, batch: 751 out of 1563
loss: 1.5641603469848633, batch: 801 out of 1563
loss: 1.9058037996292114, batch: 851 out of 1563
loss: 1.9078322649002075, batch: 901 out of 1563
loss: 1.931149959564209, batch: 951 o

In [16]:
model_name = "cnn_2"
if True:
    torch.save(model.state_dict(), MODELS_PATH + model_name + ".pth")

    with open(MODELS_PATH + model_name + ".txt", "w") as f:
        f.write("Epochs: {}\n".format(epochs))
        f.write("Feature Selection: {}\n".format("MRMR"))
        f.write("Feature Set: {}\n".format(FEATURES))
        f.write("Model: {}\n".format(str(model)))
        f.write("Loss Function: {}\n".format("Cross Entropy Loss"))
        f.write("Optimizer: {}\n\n\n\n".format(str(optimizer)))
        f.write("Results: {}\n".format(perf_acc))
        f.write("Timer: {}s\n".format(round(perf_timer, 2)))

Saved PyTorch Model State to model.pth
